$\textbf{Sample code of value iteration algorithm (Algorithm 6.1 in [1]) to find Nash Equilibrium in constant}$ $\textbf{sum, stochastic, Advanced Persistent Threats-Dynamic Information Flow Tracking (APT-DIFT) games}$

$\textbf{Code Description:}$

(1) Python Version: 3.7

(2) This code takes as input Example_Data.mat file. Example_Data.mat is a data file containing state space of APT-DIFT game for single stage attack example.

(3) For detailed explanation of the value iteration algorithm and APT-DIFT game model please refer to [1].

$\textbf{Note:}$ You may freely redistribute and use this sample code, with or without modification, provided you include the original 
Copyright notice and use restrictions.

$\textbf{Disclaimer:}$ THE SAMPLE CODE IS PROVIDED "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL DINUKA SAHABANDU OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) SUSTAINED BY YOU OR A THIRD PARTY, HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT ARISING IN ANY WAY OUT OF THE USE OF THIS SAMPLE CODE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
    
For additional information, contact:
Dinuka Sahabandu, email: sdinuka@uw.edu

$\textbf{Acknowledgement:}$ This work was supported by ONR grant N00014-16-1-2710 P00002, DARPA TC grant DARPA FA8650-15-C-7556, and ARO grant W911NF-16-1-0485.

[1] Shana Moothedath, Dinuka Sahabandu, Joey Allen, Linda Bushnell, Wenke Lee, and Radha Poovendran, “Stochastic Dynamic Information Flow Tracking Game using Supervised Learning for Detecting Advanced Persistent Threats".
     - Arxiv link: https://arxiv.org/pdf/2007.12327.pdf
     - Website: https://adapt.ece.uw.edu/

In [1]:
import numpy as np
import networkx as nx
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from numpy.random import rand
from scipy.io import loadmat

In [2]:
# Define game related parameters here (e.g., IFG, entrypoints, destinations, rewards, etc...)

#Load Nation State attack state space (without s0, \Tau_A, \Tau_B and \Phi states)
IFG_Data = loadmat('Example_Data')
IFG = IFG_Data['SS_transition_matrix']

#Define set of states corresponding to entrypoint(s) of the attack 
entry_points = np.array([26])
#Define set of states corresponding to final destination(s) of the attack 
destination = np.array([29])
#Define set of states corresponding to intermediate destinaton(s) of the attack 
#intermediate_destinations = np.array([2,9])
#Define False negatives asscoiated with the states 
FN = [0.1, 0.2, 0.05, 0.1, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1]
#Define False positives asscoiated with the states 
FP = [0.2, 0.1, 0.1, 0.05, 0.1, 0.2, 0.1, 0.1, 0.05, 0.1, 0.2, 0.1, 0.1, 0.05, 0.1, 0.2, 0.1, 0.1, 0.05, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1, 0.1, 0.2, 0.05, 0.1, 0.1]

#Define number of iterations for the value iteration algorithm 
sim_time = 1000
#Define reward value (beta) 
beta = 100
#Terminating threshold for value iteration algorithm 
delta = 1e-7

IFG_length = len(IFG)
IFG_Graph = nx.from_numpy_matrix(np.array(IFG))

IFG_graph = nx.DiGraph()
for ii in range(IFG_length):
    for jj in range(IFG_length):
        if IFG[ii, jj] == 1:
            IFG_graph.add_edge(ii, jj)



SS_length = IFG_length + 4  # s0, tau_A, tau_B, tau_C
SS = np.zeros((SS_length, SS_length))

for ii in range(SS_length - 1):
    if ii <= (IFG_length - 1) and ii != destination[0]:
        SS[ii, IFG_length + 1:IFG_length + 4] = np.ones((1, 3), dtype=int)
    if ii == IFG_length:
        SS[ii, entry_points] = 1
    for jj in range(SS_length - 1):
        if ii <= (IFG_length - 1) and jj <= (IFG_length - 1):
            SS[ii, jj] = IFG[ii, jj]

SS_graph = nx.DiGraph()
for ii in range(SS_length):
    for jj in range(SS_length):
        if SS[ii, jj] == 1:
            SS_graph.add_edge(ii, jj)

# nx.draw_networkx(IFG_graph, with_labels=True)
# plt.show()

# Define value function

V = np.zeros((sim_time, SS_length))
V[1, IFG_length + 1] = beta
V[1, IFG_length + 3] = beta

# Action set of players at IFG nondestination nodes and policy of defender
# Action sets
Action_set_DIFT = []
Action_set_APT = []
neighbors = []

for ii in range(0, IFG_length + 1):
    if (ii == IFG_length):  # Action sets at the virtual state
        Action_set_DIFT.append([-2])  #action of DIFT ---> pseudo action
        Action_set_APT.append(entry_points.tolist()) 
    else:
        neighbors = np.nonzero(IFG[ii, :])
        Action_set_DIFT.append(neighbors[0])
        Action_set_APT.append(neighbors[0])
        x = Action_set_DIFT[ii].tolist()
        x.extend([-1])  # -1 action of DIFT ---> performing a security analysis
        y = Action_set_APT[ii].tolist()
        y.extend([-1])  # -1 action of APT ---> quitting thee attack
        Action_set_DIFT[ii] = x
        Action_set_APT[ii] = y


def Value_Iteration():
    # Policy for DIFT
    DIFT_Policy = []
    Value_Data = []
    Difference_Data = []
    for ii in range(0, IFG_length):
        DIFT_Policy.append(np.zeros((len(Action_set_DIFT[ii]))).tolist())
        
    avoid = [IFG_length + 1, IFG_length + 2, IFG_length + 3]
    avoid.extend(destination)
    t = 0
    while (max(abs(V[t + 1, :] - V[t, :])) > delta):
        t = t + 1
        # print(t)
        for ii in range(0, SS_length):
            if (ii in avoid):  # Avoiding destination, \tau_A, \tau_B, \phi (No LP is solved here)
                V[t + 1, ii] = V[t, ii]
            else:  # LP is solved in these states
                if ii != IFG_length:  # Not virtual state
                    # Define Linear Program
                    # Variable structure [p(s,d1), ..., p(s,dN), V(s)]

                    # Objective function
                    obj = np.zeros(len(DIFT_Policy[ii])).tolist()
                    obj.extend([-1])  # last entry corresponding to value function at state ii

                    # Linear equality constraints (Total one)
                    # Sum of DIFT's defense policy at each state ii is one
                    lhs_eq = np.ones((1, len(DIFT_Policy[ii]) + 1)).tolist()  # +1 to represent V(s)
                    lhs_eq[0][-1] = 0  # last entry is "0" (no V(s) involved)
                    rhs_eq = [1]

                    # Linear inequality constraints (Total = # actions of DIFT at state ii + # of actions of APT at state ii
                    # Each entry is poliy of DIFT must be > 0 + For each APT's action V(s) <= \sum_{d} Q(s,d,a)pD(s,d)
                    len_lhs_ineq = len(Action_set_DIFT[ii]) + len(Action_set_APT[ii])
                    lhs_ineq = np.zeros((len_lhs_ineq, len(DIFT_Policy[ii]) + 1)).tolist()
                    for jj in range(0, len_lhs_ineq):
                        if jj < len(Action_set_DIFT[ii]):  # constraint on the probability
                            lhs_ineq[jj][jj] = -1

                        else:  # constraints on Q and DIFT's policy
                            lhs_ineq[jj][-1] = 1  # V(s) position
                            kk = jj - len(Action_set_DIFT[ii])  # APT action
                            for ll in range(0, len(Action_set_DIFT[ii])):
                                Q = 0
                                if kk == (len(Action_set_APT[ii]) - 1):  # APT dropout
                                    Q = V[t, IFG_length + 3]  # Next state is \phi
                                else:  # APT not dropped out
                                    if ll < (len(Action_set_DIFT[ii]) - 1):  # DIFT trap
                                        if kk == ll:  # Trap performed at the APT's state
                                            # Two cases due to False negatives
                                            Q = (1 - FN[Action_set_DIFT[ii][ll]]) * V[t, IFG_length + 1] + FN[
                                                Action_set_DIFT[ii][ll]] * V[t, Action_set_APT[ii][kk]]
                                            # Next state is APT's transition action +   \tau_A
                                        else:  # unsuccessful trap due to different place of trapping
                                            # Two cases due to False Positives
                                            Q = (1 - FP[Action_set_DIFT[ii][ll]]) * V[t, Action_set_APT[ii][kk]] + FP[
                                                Action_set_DIFT[ii][ll]] * V[t, IFG_length + 2]
                                            # Next state is APT's transition action +   \tau_B
                                    else:  # DIFT does not trap
                                        Q = V[t, Action_set_APT[ii][kk]]  # Next state is APT's transition action

                                lhs_ineq[jj][ll] = -Q

                    rhs_ineq = np.zeros(len_lhs_ineq).tolist()

                    # Solve LP
                    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, A_eq=lhs_eq, b_eq=rhs_eq,
                                  method="revised simplex")
                    DIFT_Policy_tmp = opt.x[0:-1]
                    # Update DIFT's Policy
                    DIFT_Policy[ii] = DIFT_Policy_tmp.tolist()
                    # Update the value function
                    V[t + 1, ii] = opt.x[-1]

                else:  # Virtual state
                    tmp_s0_value = beta
                    for nn in range(0, len(entry_points)):
                        #print('Compare tmp vs entry val',tmp_s0_value,V[t + 1, entry_points[nn]])
                        if tmp_s0_value > V[t + 1, entry_points[nn]]:
                            tmp_s0_value = V[t + 1, entry_points[nn]]
                            #print('Set tmp:',tmp_s0_value)
                        
                    V[t + 1, IFG_length] = tmp_s0_value


        print('-----------------Value Iteration Results----------------')
        print('Iteration Number:',t,'Difference',max(abs(V[t + 1, :] - V[t, :])))
        print('Values at states',V[t+1,:])
        print('--------------------------------------------------------')
        Value_Data.append(V[t,entry_points[0]])
        Difference_Data.append(max(abs(V[t + 1, :] - V[t, :])))
    print('----------------------------------DIFT_Policy-------------------------------------------')
    print(DIFT_Policy)
    print('----------------------------------Value_Data-------------------------------------------')
    print(Value_Data)    
    print('----------------------------------Difference_Data-------------------------------------------')
    print(Difference_Data)
    
def main():
    Value_Iteration()

if __name__ == "__main__":
    main()


-----------------Value Iteration Results----------------
Iteration Number: 1 Difference 100.0
Values at states [ 22.10016155  12.95454545  42.35294118 100.          43.42857143
 100.          90.          95.          30.53571429  42.35294118
  90.          80.          22.8         14.64668094  22.10016155
  80.          90.          14.52229299  17.92922674  17.74319066
  90.          17.74319066  30.53571429  46.21621622 100.
  30.53571429  90.          90.          90.           0.
  90.         100.           0.         100.        ]
--------------------------------------------------------
-----------------Value Iteration Results----------------
Iteration Number: 2 Difference 41.400364988088086
Values at states [ 35.44855221  54.35491044  82.97362794 100.          50.48394652
 100.          91.77431907  97.11764706  46.17622131  59.13196898
  94.62162162  82.59090909  46.17622131  43.86186719  35.44855221
  98.          99.          42.52670839  35.44855221  35.44855221
  92.21001

-----------------Value Iteration Results----------------
Iteration Number: 21 Difference 0.00014374274465467352
Values at states [ 74.60682051  85.296659    90.62825286 100.          85.81612257
 100.          97.46066768  99.5314093   74.60682051  86.71502547
  98.29688402  97.05930977  74.60682051  74.60682051  74.60682051
 100.         100.          77.33504348  74.60682051  74.60682051
  97.46066768  74.60682051  74.60682051  82.96892871 100.
  74.60682051  98.29688402  90.         100.           0.
  98.29688402 100.           0.         100.        ]
--------------------------------------------------------
-----------------Value Iteration Results----------------
Iteration Number: 22 Difference 7.841697726007624e-05
Values at states [ 74.60689893  85.29671786  90.628287   100.          85.81618501
 100.          97.46068205  99.53141264  74.60689893  86.71507861
  98.29689287  97.0593318   74.60689893  74.60689893  74.60689893
 100.         100.          77.33511956  74.60689893  